In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 학습 데이터
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 테스트 데이터
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=50, shuffle=True)

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 4 * 4 * 50) # [batch_size, 50, 4, 4]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
cnn = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

In [8]:
train_loader.

In [5]:
cnn.train()  # 학습을 위함
for epoch in range(10):
    for index, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()  # 기울기 초기화
        output = cnn(data)
        loss = criterion(output, target)
        loss.backward()  # 역전파
        optimizer.step()

        if index % 100 == 0:
            print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))

loss of 0 epoch, 0 index : 2.303093910217285
loss of 0 epoch, 100 index : 1.0741589069366455
loss of 0 epoch, 200 index : 0.40458378195762634
loss of 0 epoch, 300 index : 0.454990416765213
loss of 0 epoch, 400 index : 0.3319699764251709
loss of 0 epoch, 500 index : 0.17312313616275787
loss of 0 epoch, 600 index : 0.24067924916744232
loss of 0 epoch, 700 index : 0.33306530117988586
loss of 0 epoch, 800 index : 0.262826144695282
loss of 0 epoch, 900 index : 0.29862239956855774
loss of 0 epoch, 1000 index : 0.16253280639648438
loss of 0 epoch, 1100 index : 0.1255272924900055
loss of 1 epoch, 0 index : 0.03797267749905586
loss of 1 epoch, 100 index : 0.03237491101026535
loss of 1 epoch, 200 index : 0.16908065974712372
loss of 1 epoch, 300 index : 0.06191662326455116
loss of 1 epoch, 400 index : 0.11565271019935608
loss of 1 epoch, 500 index : 0.07763880491256714
loss of 1 epoch, 600 index : 0.09656323492527008
loss of 1 epoch, 700 index : 0.12894965708255768
loss of 1 epoch, 800 index : 0.

In [6]:
cnn.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = cnn(data)
        test_loss += criterion(output, target).item() # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Test set: Average loss: 6.4592, Accuracy: 9884/10000 (99%)

